In [2]:
from pymilvus import MilvusClient 
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import copy
from tqdm import tqdm 
from dotenv import load_dotenv 
load_dotenv('key.env', override=True)

True

In [2]:
client = MilvusClient(uri="medical.db")
collection_name = "medical"

In [3]:
if client.has_collection(collection_name):
    client.drop_collection(collection_name)

In [3]:
book = PyPDFLoader('BOA-and-BAPRAS-Standards-for-Open-fractures-2020.pdf')

In [4]:
pdf_pages = book.load()

In [5]:
pdf_pages

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'creationdate': '2020-07-25T14:48:13+05:30', 'moddate': '2020-09-16T16:19:29+01:00', 'title': 'PDF generated by "Newgen_sugumar"', 'trapped': '/False', 'source': 'BOA-and-BAPRAS-Standards-for-Open-fractures-2020.pdf', 'total_pages': 256, 'page': 0, 'page_label': 'i'}, page_content='Standards for\xa0the \nManagement of\xa0  \nOpen Fractures\nThis is an open access publication. Except where otherwise noted, this work is distributed under the terms of a Creative Commons Attribution-NonCommercial \n4.0 International licence. For enquiries concerning use outside the scope of the licence terms, please contact academic.permissions@oup.com'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'creationdate': '2020-07-25T14:48:13+05:30', 'moddate': '2020-09-16T16:19:29+01:00', 'title': 'PDF generated by "Newgen_sugumar"', 'trapped': '/Fal

In [6]:
len(pdf_pages)

256

In [7]:
pages_pdf = copy.deepcopy(pdf_pages)

In [8]:
pages_pdf[12].page_content

'Contributors\nA central theme of this book is collaboration. It is itself a manifestation of an \northoplastic team working to a common purpose. Its strength comes from the \ngroup responsibility taken for the efforts of the individual contributors; hence \nan absence of names on the front cover and at the beginning of chapters. While \nall the authors were able to review, discuss, and agree every chapter to broaden \nthe review of the evidence, there were lead contributors to each chapter who did \nsignificant work that should be recognized. Listed below are the contributors \nto this book in alphabetical order. By each name is noted the chapter numbers \nwhere they were a lead author; additionally if they were involved in producing \nthe relevant NICE Complex Fracture Guidelines or on the BOA Trauma group \n(the origin of the Open Fracture BOAST).\nAlexander Aquilina, BSc, MB ChB, MRCS, MSc\nClinical Research Fellow and DPhil candidate\nOxford Trauma Group\nNuffield Department of Or

In [9]:
' '.join(pages_pdf[12].page_content.split())

'Contributors A central theme of this book is collaboration. It is itself a manifestation of an orthoplastic team working to a common purpose. Its strength comes from the group responsibility taken for the efforts of the individual contributors; hence an absence of names on the front cover and at the beginning of chapters. While all the authors were able to review, discuss, and agree every chapter to broaden the review of the evidence, there were lead contributors to each chapter who did significant work that should be recognized. Listed below are the contributors to this book in alphabetical order. By each name is noted the chapter numbers where they were a lead author; additionally if they were involved in producing the relevant NICE Complex Fracture Guidelines or on the BOA Trauma group (the origin of the Open Fracture BOAST). Alexander Aquilina, BSc, MB ChB, MRCS, MSc Clinical Research Fellow and DPhil candidate Oxford Trauma Group Nuffield Department of Orthopaedics, Rheumatology 

In [10]:
for page in pages_pdf:
    page.page_content = ' '.join(page.page_content.split())

In [11]:
char_splitter = CharacterTextSplitter(
   separator=".",
   chunk_size=500,
   chunk_overlap=50 
)

In [12]:
pages_splitted = char_splitter.split_documents(pages_pdf)

Created a chunk of size 554, which is longer than the specified 500
Created a chunk of size 976, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 594, which is longer than the specified 500
Created a chunk of size 597, which is longer than the specified 500
Created a chunk of size 1464, which is longer than the specified 500
Created a chunk of size 887, which is longer than the specified 500
Created a chunk of size 931, which is longer than the specified 500
Created a chunk of size 1111, which is longer than the specified 500
Created a chunk of size 1253, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500
Created a chunk of size 538, which is longer than the specified 500
Created a chunk of size 758, which is longer than the specified 500
Created a chunk of size 527, which is longer than the specified 500
Created a chunk of size 2486, which is longer

In [13]:
len(pages_splitted)

1425

In [14]:
pages_splitted

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'creationdate': '2020-07-25T14:48:13+05:30', 'moddate': '2020-09-16T16:19:29+01:00', 'title': 'PDF generated by "Newgen_sugumar"', 'trapped': '/False', 'source': 'BOA-and-BAPRAS-Standards-for-Open-fractures-2020.pdf', 'total_pages': 256, 'page': 0, 'page_label': 'i'}, page_content='Standards for the Management of Open Fractures This is an open access publication. Except where otherwise noted, this work is distributed under the terms of a Creative Commons Attribution-NonCommercial 4.0 International licence. For enquiries concerning use outside the scope of the licence terms, please contact academic.permissions@oup.com'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'creationdate': '2020-07-25T14:48:13+05:30', 'moddate': '2020-09-16T16:19:29+01:00', 'title': 'PDF generated by "Newgen_sugumar"', 'trapped': '/False', 'source': 

In [15]:
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024
)

In [18]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma(
    collection_name="medical",
    embedding_function=embedding,
    persist_directory="./medical_db",
)

/var/folders/4c/c9068fm57wx9ky5bbw7nw9c40000gn/T/ipykernel_49057/3480146804.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [19]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_splitted))]
vector_store.add_documents(documents=pages_splitted, ids=uuids)

['75770c16-602a-4cf8-833a-4e8fc6c84d52',
 '87f7722e-3eec-4b85-9e7f-d000c075137a',
 '5ecd3be0-70db-474e-9efc-0f6de13900f6',
 'b3d48c9a-9e4d-408a-9391-614e2d9ea168',
 'ab69a9e4-877c-4684-ab87-b63fe62b923b',
 'ae080dc8-5cf7-45bc-8892-e673c65db956',
 '7077cf90-0dc5-4d38-a074-b1a8e7e536f7',
 '6f2e5e7e-a12a-42cb-a9c6-8b0d2b53942c',
 '274d2de2-0845-4b5c-8c90-ee2cb25ee2b6',
 '0da88208-6a0d-4db0-b097-9e608330c94e',
 '90a9571a-19f5-4c7a-bff0-e806bb1752b8',
 'fa396446-bc8f-49ac-9e11-83ebc63faaea',
 'd27e2060-1603-4d0d-838d-f7611a1cd614',
 'c9ac129b-2655-42cd-9bd3-bd44475c7093',
 '7b0b6b6e-ca1f-463f-993e-736259165d06',
 '4fb0f544-eed9-4d78-8866-73764ec1f716',
 '295f5059-79e7-4437-9151-d4c3a041630d',
 'ed80c1c6-cfed-43c7-960f-88c55a4fc913',
 'aa33f11e-1ee0-406b-a53d-068807465ae0',
 'a5ac4d3e-5b7f-4d7c-ac10-154fb9eecab3',
 '32f7c169-32f4-4689-a27f-809a4e67640a',
 '199a8f5d-8409-405e-8263-aac5588a720e',
 '4a3a9e23-accc-4b03-9cd5-dcbae35d4295',
 'e344f2ca-fe79-4d1e-939d-df080e70a608',
 '7a5c337f-9a9b-